**시계열 예측을 다루는 통계적 기법** 중 가장 널리 알려진 **ARIMA(Auto-regressive Integrated Moving Average)**에 대해 알아보고, 이를 토대로 '특정 주식 종목의 가격을 예측해보는 실습'을 진행한다.

<br>

그 외에도 시계열 예측에 사용되는 모델은 Prophet, LSTM 등 딥러닝을 활용하는 방법도 있다. 하지만 ARIMA는 탄탄한 통계학 이론적 기반을 갖추고 있으므로 시계열 데이터를 이해하는 관점을 훈련하는 데 도움이 될 것이다.


<br>

## 학습 목표
- 시계열 데이터의 특성과 안정적(Stationary) 시계열의 개념을 이해한다.
- **ARIMA 모델을 구성하는 AR, MA, Diffencing**의 개념을 이해하고 간단한 시계열 데이터에 적용해 본다.
- 실제 주식 데이터에 ARIMA를 적용해서 예측 정확도를 확인해 본다.

<br>

